# Premier League Prediction

### Libraries

In [3]:
import pandas as pd

### Initialization

In [4]:
matches = pd.read_csv("../data/premier_league_data.csv")
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2015-08-08,15:00,Premier League,Matchweek 1,Sat,Home,W,4,2,Sunderland,...,Match Report,NaN,20.0,8.0,NaN,0,1,1.0,2023,Leicester City
1,2015-08-15,15:00,Premier League,Matchweek 2,Sat,Away,W,2,1,West Ham,...,Match Report,NaN,10.0,5.0,NaN,0,0,0.0,2023,Leicester City
2,2015-08-22,15:00,Premier League,Matchweek 3,Sat,Home,D,1,1,Tottenham,...,Match Report,NaN,13.0,2.0,NaN,0,0,0.0,2023,Leicester City
3,2015-08-29,15:00,Premier League,Matchweek 4,Sat,Away,D,1,1,Bournemouth,...,Match Report,NaN,4.0,2.0,NaN,0,1,1.0,2023,Leicester City
4,2015-09-13,16:00,Premier League,Matchweek 5,Sun,Home,W,3,2,Aston Villa,...,Match Report,NaN,21.0,6.0,NaN,0,0,0.0,2023,Leicester City


In [5]:
matches.shape

(9880, 26)

In [10]:
38 * 20 * 13

9880

In [7]:
matches['team'].value_counts()

team
Arsenal                    494
Tottenham Hotspur          494
Manchester United          494
Manchester City            494
Chelsea                    494
Liverpool                  494
Everton                    494
Aston Villa                494
Newcastle United           456
Fulham                     418
West Ham United            380
Sunderland                 380
Bolton Wanderers           342
Blackburn Rovers           342
West Bromwich Albion       342
Stoke City                 304
Wigan Athletic             304
Portsmouth                 266
Birmingham City            228
Southampton                228
Middlesbrough              228
Swansea City               190
Norwich City               190
Hull City                  152
Charlton Athletic          152
Crystal Palace             152
Wolverhampton Wanderers    152
Leicester City             114
Reading                    114
Queens Park Rangers        114
Watford                     76
Burnley                     76
Bou

In [13]:
print("In 13 seasons, top teams played ", 13*38, "matches")

In 13 seasons, top teams played  494 matches


In [8]:
matches["round"].value_counts()

round
Matchweek 1     260
Matchweek 2     260
Matchweek 3     260
Matchweek 4     260
Matchweek 5     260
Matchweek 6     260
Matchweek 7     260
Matchweek 8     260
Matchweek 9     260
Matchweek 10    260
Matchweek 11    260
Matchweek 12    260
Matchweek 13    260
Matchweek 14    260
Matchweek 15    260
Matchweek 16    260
Matchweek 17    260
Matchweek 18    260
Matchweek 19    260
Matchweek 20    260
Matchweek 21    260
Matchweek 22    260
Matchweek 23    260
Matchweek 24    260
Matchweek 25    260
Matchweek 26    260
Matchweek 27    260
Matchweek 28    260
Matchweek 29    260
Matchweek 30    260
Matchweek 31    260
Matchweek 32    260
Matchweek 33    260
Matchweek 34    260
Matchweek 35    260
Matchweek 36    260
Matchweek 37    260
Matchweek 38    260
Name: count, dtype: int64

In [11]:
matches['season'].value_counts()

season
2023    760
2022    760
2021    760
2020    760
2019    760
2018    760
2017    760
2016    760
2015    760
2014    760
2013    760
2012    760
2011    760
Name: count, dtype: int64